In [1]:
using Base
using StaticArrays

In [11]:
struct Dual{T}
    val::T
    grad::T
end

Base.:+(f::Dual, g::Dual) = Dual(f.val+g.val, f.grad+g.grad)
Base.:+(f::Dual, g::Real) = Dual(f.val+g, f.grad)
Base.:+(f::Real, g::Dual) = Dual(f+g.val, g.grad)

Base.:*(f::Dual, g::Dual) = Dual(f.val*g.val, f.val*g.grad+f.grad*g.val)
Base.:*(f::Dual, g::Real) = Dual(f.val*g, f.grad*g)
Base.:*(f::Real, g::Dual) = Dual(f*g.val, f*g.grad)


struct MultiDual{N,T}
	val::T
	# SVector is static vector which lives on the stack
	grads::SVector{N,T} 
end

function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val+g.val, f.grads+g.grads)
end

function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val*g.val, f.val*g.grads+f.grads*g.val)
end

function Base.:^(f::MultiDual{N,T}, g::Real) where {N,T}
    return Base.power_by_squaring(f,g)
end

In [18]:
arr = rand(3)
out = zeros(1)

test = 2.0*Dual(arr[1], 1.0) + 3.0*Dual(arr[2], 0.0) + 4.0*Dual(arr[3], 0.0)

println(arr)
println(test)

[0.18154965230087694, 0.49662287213541156, 0.34701580613435246]
Dual{Float64}(3.2410311455453984, 3.0)


In [12]:
ff(x, y) = x^2 + x*y
gg(x, y) = y^3 + x

# Jacobian at x=3 and y=4
xx = MultiDual(3., SVector(1.,0.))
yy = MultiDual(4., SVector(0.,1.))

println("Jacobian: ", ff(xx, yy).grads, gg(xx, yy).grads)

MethodError: MethodError: convert(::Type{Union{}}, ::MultiDual{2, Float64}) is ambiguous.

Candidates:
  convert(::Type{T}, arg) where T<:VecElement
    @ Base baseext.jl:19
  convert(T::Type{<:Core.IntrinsicFunction}, x)
    @ Base essentials.jl:289
  convert(T::Type{<:Nothing}, x)
    @ Base essentials.jl:290

Possible fix, define
  convert(::Type{Union{}}, ::Any)
